In [66]:
import re
import requests
import numpy as np
from bs4 import BeautifulSoup
import itertools
import polars as pl

In [111]:
def toilet_info(URL: str):
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")

    #find name of toilet
    h2 = str(soup.find_all("h2", class_="text-3xl font-extrabold text-gray-900"))
    name = re.findall("<h2[^>]*>([^<]+)</h2>", h2)

    #find address of toilet
    p = str(soup.findAll("p", class_="text-sm text-gray-500 truncate mb-2 uppercase flex flex-row"))
    address = re.findall("<span[^>]*>([^<]+)</span>", p)
    address[1] = address[1].capitalize()

    #find lat and long of toilet
    script = str(soup.find_all("script"))
    location = re.findall(".+?(?<=setView\(\[)(.\d*\.?\d*)\,(..\d*\.?\d*)", script)

    #rating
    span = str(soup.find_all("span", class_="text-sm"))
    rating = re.findall("\((.+?)\)", span)

    #facilities
    repg = str(soup.find_all("span", class_="px-2"))
    facilities = re.findall("<span[^>]*>([^<]+)</span>", repg)

    return {
        "name": name[0],
        "address": ''.join(address),
        "tags": ", ".join(facilities),
        "lat": location[0][0],
        "lng": location[0][1],
        "rating": rating[0][0]
    }

<>:17: SyntaxWarning: invalid escape sequence '\('
<>:21: SyntaxWarning: invalid escape sequence '\('
<>:17: SyntaxWarning: invalid escape sequence '\('
<>:21: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_5641/2050576992.py:17: SyntaxWarning: invalid escape sequence '\('
  location = re.findall(".+?(?<=setView\(\[)(.\d*\.?\d*)\,(..\d*\.?\d*)", script)
/tmp/ipykernel_5641/2050576992.py:21: SyntaxWarning: invalid escape sequence '\('
  rating = re.findall("\((.+?)\)", span)


In [35]:
toilets = []

for page_num in range(515):
    url = "https://mywc.kpkt.gov.my/?keyword=&state_id=&page={}#toilet-search-list".format(page_num)
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    links = soup.find_all('a')
    urls = [link.get('href') for link in links]
    toilet_filter = filter(lambda s: "https://mywc.kpkt.gov.my/toilet/" in s, urls)
    toilets += [t for t in toilet_filter]

In [116]:
names = []
addresses = []
tags = []
lats = []
lngs = []
ratings = []

for toilet in toilets[:50]:
    info = toilet_info(toilet)
    names.append(info['name'])
    addresses.append(info['address'])
    tags.append(info['tags'])
    lats.append(info['lat'])
    lngs.append(info['lng'])
    ratings.append(info['rating'])


In [125]:
df = pl.DataFrame({
    "url": toilet[:50],
    "name": names,
    "address": addresses,
    "tags": tags,
    "lat": lats,
    "lng": lngs,
    "rating": ratings
})

df

url,name,address,tags,lat,lng,rating
str,str,str,str,str,str,str
"""https://mywc.kpkt.gov.my/toile…","""Medan Selera Seksyen 16, Banda…","""Jalan 6C/5, Selangor""","""Mesra OKU, 24 Jam""",""" 2.9429730""",""" 101.7644010""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Medan Selera Parit""","""Medan Selera Parit, Perak""","""Mesra OKU, 24 Jam""",""" 4.4769680""",""" 100.9081940""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Pasar Sungkai""","""sungkai, Perak""","""Mesra OKU, Berbayar""",""" 4.0006700""",""" 101.1820900""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Medan Selera Bidor""","""Bidor, Perak""","""Berbayar""",""" 4.0534200""",""" 101.1654900""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Tandas Awam KPKT""","""Jalan Bukit Tinggi, Johor""","""Mesra OKU, Berbayar, Tempat Me…",""" 3.3977100""",""" 101.7679000""","""5"""
…,…,…,…,…,…,…
"""https://mywc.kpkt.gov.my/toile…","""Medan Selera Seri Iskandar""","""Seri Iskandar, Perak""","""24 Jam""",""" 4.3594510""",""" 100.9633870""","""3"""
"""https://mywc.kpkt.gov.my/toile…","""Hentian Rehat Perah""","""Hentian Rehat Perah, Perak""","""Berbayar, 24 Jam""",""" 5.4509680""",""" 101.1581210""","""3"""
"""https://mywc.kpkt.gov.my/toile…","""Pasar Bidor""","""Bidor, Perak""","""Mesra OKU, Berbayar""",""" 4.0637000""",""" 101.1711300""","""3"""


In [126]:
df.filter(pl.col('tags').str.contains('Mesra OKU'))

url,name,address,tags,lat,lng,rating
str,str,str,str,str,str,str
"""https://mywc.kpkt.gov.my/toile…","""Medan Selera Seksyen 16, Banda…","""Jalan 6C/5, Selangor""","""Mesra OKU, 24 Jam""",""" 2.9429730""",""" 101.7644010""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Medan Selera Parit""","""Medan Selera Parit, Perak""","""Mesra OKU, 24 Jam""",""" 4.4769680""",""" 100.9081940""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Pasar Sungkai""","""sungkai, Perak""","""Mesra OKU, Berbayar""",""" 4.0006700""",""" 101.1820900""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Tandas Awam KPKT""","""Jalan Bukit Tinggi, Johor""","""Mesra OKU, Berbayar, Tempat Me…",""" 3.3977100""",""" 101.7679000""","""5"""
"""https://mywc.kpkt.gov.my/toile…","""Taman Tasik Cempaka, Bandar Ba…","""Seksyen 9, Bandar Baru Bangi, …","""Mesra OKU""",""" 2.9604490""",""" 101.7574640""","""4"""
…,…,…,…,…,…,…
"""https://mywc.kpkt.gov.my/toile…","""Dataran Iskandar Perdana""","""Dataran Iskandar Perdana, Pera…","""Mesra OKU, 24 Jam""",""" 4.3611580""",""" 100.9747480""","""3"""
"""https://mywc.kpkt.gov.my/toile…","""Wisma MDK""","""Tandas Wisma MDK, Perak""","""Mesra OKU, 24 Jam""",""" 5.1299900""",""" 100.4894600""","""3"""
"""https://mywc.kpkt.gov.my/toile…","""Pasar Bidor""","""Bidor, Perak""","""Mesra OKU, Berbayar""",""" 4.0637000""",""" 101.1711300""","""3"""


In [124]:
df.write_csv("toilets2.csv")

In [74]:
df_toilet = pl.DataFrame(toilets, schema={"url": pl.String})

In [77]:
df_toilet

url
str
"""https://mywc.kpkt.gov.my/toile…"
"""https://mywc.kpkt.gov.my/toile…"
"""https://mywc.kpkt.gov.my/toile…"
"""https://mywc.kpkt.gov.my/toile…"
"""https://mywc.kpkt.gov.my/toile…"
…
"""https://mywc.kpkt.gov.my/toile…"
"""https://mywc.kpkt.gov.my/toile…"
"""https://mywc.kpkt.gov.my/toile…"


In [100]:
df_toilet = df_toilet.with_columns(
    pl.lit("").alias('name'),
    pl.lit("").alias('location'),
    pl.lit("").alias('tags'),
    n_stars = 0,
    lat = 0.0,
    lng = 0.0
)

In [107]:
for row in df_toilet.iter_rows(named=True):
    row['lat'] = 5.0
    print(row['lat'])

5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0


In [108]:
df_toilet

url,name,location,tags,n_stars,lat,lng
str,str,str,str,i32,f64,f64
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
…,…,…,…,…,…,…
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0
"""https://mywc.kpkt.gov.my/toile…","""""","""""","""""",0,0.0,0.0


In [65]:
df_toilet.write_csv('toilets.csv')